In [3]:
### header ###
__author__ = "Jenhan Tao"
__license__ = "MIT"
__email__ = "jenhantao@gmail.com"

### imports ###
import sys # system functions
import os # os functions
import pandas as pd # for reading data
import numpy as np # for numeric operations
import matplotlib.pyplot as plt 
import matplotlib # for visualizing data
import scipy # scientific computing
import seaborn as sns # for pretty plots

### notebook specific configuration ###
%matplotlib inline
matplotlib.rcParams['savefig.dpi'] = 200
sns.set_context('notebook')
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
# it's bad practice to work within a github repo and also bad practice to have data in a github repo
# but it's convenient
# change working directory to github repo
working_directory = '/gpfs/data01/glasslab/home/jtao/side_projects/fantasy_football_genetic_algorithm_draft/'
os.chdir(working_directory)

### Read in rankings

In [17]:
year_rankFrame_dict = {} # {year:DataFrame}
for f in os.listdir(working_directory + '/rankings_standard'):
    json_path = working_directory + '/rankings_standard/' + f
    ranks = pd.read_json(json_path, typ='series')
    adp_list = [x['adp'] for x in ranks['players']]
    name_list = [x['name'] for x in ranks['players']]
    position_list = [x['position'] for x in ranks['players']]
    
    year = f.split('_')[-1].replace('.json','')
    
    rankFrame = pd.DataFrame({'ADP':adp_list, 'Position':position_list, 'Name':name_list})
    year_rankFrame_dict[year] = rankFrame
    


In [18]:
rankFrame.head()

,ADP,Name,Position
0,1.6,LeSean McCoy,RB
1,3.1,Adrian Peterson,RB
2,3.5,Jamaal Charles,RB
3,4.2,Matt Forte,RB
4,5.5,Calvin Johnson,WR


### Read in season results

In [28]:
year_scoreFrame_dict = {} # {year:DataFrame}
for f in os.listdir(working_directory + '/scoring_standard'):
    score_path = working_directory + '/scoring_standard/' + f
    scoreFrame = pd.read_csv(score_path, sep=',')
    year_scoreFrame_dict[year] = scoreFrame
    


In [29]:
scoreFrame.head()

,Player,Points
0,Drew Brees,424
1,Aaron Rodgers,411
2,Tom Brady,397
3,Peyton Manning,380
4,Matt Ryan,357


### Create Players

In [49]:
position_index_dict = dict(zip(sorted(rankFrame['Position'].unique()), 
                               range(len(rankFrame['Position'].unique()))))
index_position_dict = dict(zip(range(len(rankFrame['Position'].unique())),
                                sorted(rankFrame['Position'].unique()), 
                               ))

In [48]:
position_index_dict # gives the position and the corresponding index

{'DEF': 0, 'PK': 1, 'QB': 2, 'RB': 3, 'TE': 4, 'WR': 5}

In [50]:
index_position_dict # gives the index and the corresponding index

{0: 'DEF', 1: 'PK', 2: 'QB', 3: 'RB', 4: 'TE', 5: 'WR'}

In [38]:
def create_new_players(num_players, num_rounds):
    '''
    inputs: number of players and rounds
    outputs: list of fantasy players
    '''
    num_positions = 6 # number of positions
    player_list = []
    for player_number in range(num_players):
        new_player = []
        for round_number in range(num_rounds):
            weights = np.random.random(num_positions) # initialize random weights
            normed_weights = weights/np.sum(weights) # normalize so weights sum to 1
            new_player.append(normed_weights)
    
        player_list.append((new_player))
    return np.array(player_list)

In [30]:
len(rankFrame['Position'].unique()) # number of positions

6

In [40]:
fantasy_players = create_new_players(12, 10)

In [41]:
fantasy_players.shape # first index gives the player
                      # second index gives the round
                      # third index gives the position 


(12, 10, 6)

In [ ]:
fantasy_players[0,0,0] #gives the importance for player 1 in the first round for a defense 
fantasy_players[0,0,1] #gives the importance for player 1 in the first round for a place kicker (PK)
fantasy_players[8,3,2] #gives the importance for player 9 in the fourth round for a QB

### Run Simulation

In [ ]:
def simulate_draft(rankFrame, fantasy_players, num_rounds):
    '''
    returns a list of rosters for each fantasy player
    '''
    
    return fantasy_teams

In [ ]:
def score_roster(score_frame, fantasy_teams):
    '''
    returns a list of performances for each fantasy player's team
    '''
    
    return fantasy_team_performances